## Pre-requisites

Register a Google account at [https://code.earthengine.google.com](https://code.earthengine.google.com).

In [2]:
!pip install earthengine-api

  Using cached earthengine_api-0.1.332-py3-none-any.whl
  Using cached google_api_python_client-2.66.0-py2.py3-none-any.whl (10.5 MB)
  Using cached httplib2-0.21.0-py3-none-any.whl (96 kB)
  Using cached google_auth_httplib2-0.1.0-py2.py3-none-any.whl (9.3 kB)
  Using cached google_cloud_storage-2.6.0-py2.py3-none-any.whl (105 kB)
  Using cached google_auth-2.14.1-py2.py3-none-any.whl (175 kB)
  Using cached future-0.18.2-py3-none-any.whl
  Using cached google_api_core-2.10.2-py3-none-any.whl (115 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl (10 kB)
  Using cached protobuf-4.21.9-cp37-abi3-manylinux2014_x86_64.whl (408 kB)
  Using cached googleapis_common_protos-1.57.0-py2.py3-none-any.whl (217 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached cachetools-5.2.0-py3-none-any.whl (9.3 kB)
  Using cached google_resumable_media-2.4.0-py2.py3-none-any.whl (77 kB)
  Using cached google_cloud_core-2.3.2-py2.py3-none-any.whl (29 kB)
  Using cache

TQDM Library - Progression Bar

In [3]:
!pip install tqdm

  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
You should consider upgrading via the '/usr/local/bin/python3.7 -m pip install --upgrade pip' command.


## Google Drive

## Imports and Earth Engine Setup

In [4]:
from __future__ import annotations

import math
from typing import Any, Optional

import ee
import pandas as pd

from preprocessing import ee_utils

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Authentication with the Earth Engine API

In [5]:
ee.Authenticate()

Enter verification code:  4/1AfgeXvsV7nDukcJ3Bv7zyYMCGOZwN9EmICI9yA2KeXWo01xBAZnNJ2mdZDw



Successfully saved authorization token.


In [6]:
ee.Initialize()

## Projection

In [7]:
MERCATOR_CRS = "EPSG:3857"   # Mercator projection CRS.
#METRIC_CRS = "EPSG:32634"   # Metric CRS. Useful for calculations
METRIC_CRS = {"proj":"cea"}
WGS84_CRS = "EPSG:4326"      # World Geodetic System
SIRGAS_CRS = "EPSG:4674"     # SIRGAS 2000, a projection for LATAM

## Constants

In [8]:
# ========== ADAPT THESE PARAMETERS ==========

# To export to Google Drive, uncomment the next 2 lines
EXPORT = 'drive'
BUCKET = None

# export location parameters
# Here we have to specify the full drive path, otherwise it will save using
# the root directory.
DHS_EXPORT_FOLDER = 'drive_igor/nexus_tfrecords_raw'
# This Drive folder was used to store all the Clusters recieved by the EE API

# Set CHUNK_SIZE to None to export a single TFRecord file per (country, year). However,
# this may fail if it exceeds Google Earth Engine memory limits. Decrease CHUNK_SIZE
# to a small number (<= 50) until Google Earth Engine stops reporting memory errors
CHUNK_SIZE = 30

In [9]:
# input data paths
DHS_CSV_PATH = '/root/Datasets/SelectedClusters/dataset_clean.csv'

# band names
MS_BANDS = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP1']

# image parameters
PROJECTION = WGS84_CRS    # see Projection Section above.
SCALE = 30                # export resolution: 30m/px
EXPORT_TILE_RADIUS = 127  # image dimension = (2*EXPORT_TILE_RADIUS) + 1 = 255px

## Exporting Images

In [10]:
def export_images(df: pd.DataFrame,
                  sector_type: str,
                  year: int,
                  export_folder: str,
                  chunk_size: Optional[int] = None
                  ) -> dict[tuple[str, str, int, int], ee.batch.Task]:
    '''
    Args
    - df: pd.DataFrame, contains columns ['lat', 'lon', 'sector_type', 'year']
    - sector_type: str, together with `year` determines the survey to export
    - year: int, together with `sector_type` determines the survey to export
    - export_folder: str, name of folder for export
    - chunk_size: int, optionally set a limit to the # of images exported per TFRecord file
        - set to a small number (<= 50) if Google Earth Engine reports memory errors

    Returns: dict, maps task name tuple (export_folder, sector_type, year, chunk) to ee.batch.Task
    '''
    subset_df = df[(df['country'] == sector_type) & (df['year'] == year)].reset_index(drop=True)
    if chunk_size is None:
        chunk_size = len(subset_df)
    num_chunks = int(math.ceil(len(subset_df) / chunk_size))
    tasks = {}

    for i in range(num_chunks):
        chunk_slice = slice(i * chunk_size, (i+1) * chunk_size - 1)  # df.loc[] is inclusive
        fc = ee_utils.df_to_fc(subset_df.loc[chunk_slice, :])
        start_date, end_date = ee_utils.surveyyear_to_range(year)

        # create 3-year Landsat composite image
        roi = fc.geometry()
        imgcol = ee_utils.LandsatSR(roi, start_date=start_date, end_date=end_date).merged
        imgcol = imgcol.map(ee_utils.mask_qaclear).select(MS_BANDS)
        img = imgcol.median()

        # add nightlights, latitude, and longitude bands
        img = ee_utils.add_latlon(img)
        img = img.addBands(ee_utils.composite_nl(year))

        fname = f'{sector_type}_{year}_{i:02d}'
        # tasks[(export_folder, sector_type, year, i)] = ee_utils.get_array_patches(
        #     img=img, scale=SCALE, ksize=EXPORT_TILE_RADIUS,
        #     points=fc,
        #     prefix=export_folder, fname=fname,
        #     )
        tasks[(export_folder, sector_type, year, i)] = ee_utils.get_array_patches(
            img=img, scale=SCALE, ksize=EXPORT_TILE_RADIUS,
            points=fc, export=EXPORT,
            prefix=export_folder, fname=fname,
            bucket=BUCKET)
    return tasks

In [11]:
tasks: dict[tuple[str, str, int, int], ee.batch.Task] = {}

## Verifying workflow 
We are going to drop 'literacy' and 'longevity' and 'TIPO' and renaming 'income' to 'wealthpooled' 

In [ ]:
dhs_df = pd.read_csv(DHS_CSV_PATH, float_precision='high')
# dhs_df = dhs_df.head(50)
# dhs_df.to_csv("dataset_test.csv", index = False)

In [13]:
dhs_df = dhs_df.drop(columns = ['literacy','longevity','TYPE'])
dhs_df = dhs_df.rename(columns = {'income':'wealthpooled'})

In [14]:
dhs_df.shape

(20438, 5)

In [15]:
dhs_surveys = list(dhs_df.groupby(['country', 'year']).groups.keys())
dhs_surveys

[('brazil', 2010)]

## Downloading the Images
Creating the tasks that export the images to Google Drive

In [16]:
for sector_type, year in dhs_surveys:
    new_tasks = export_images(
        df=dhs_df, sector_type=sector_type, year=year,
        export_folder=DHS_EXPORT_FOLDER, chunk_size=CHUNK_SIZE)
    tasks.update(new_tasks)

In [17]:
ee_utils.wait_on_tasks(tasks, poll_interval=60)

  0%|          | 2/682 [00:00<02:28,  4.57it/s]

Task ('drive_igor/nexus_tfrecords_raw', 'brazil', 2010, 0) finished in 0 min with state: COMPLETED
Task ('drive_igor/nexus_tfrecords_raw', 'brazil', 2010, 1) finished in 0 min with state: COMPLETED


  0%|          | 3/682 [00:00<02:23,  4.73it/s]

Task ('drive_igor/nexus_tfrecords_raw', 'brazil', 2010, 2) finished in 0 min with state: COMPLETED


  1%|          | 5/682 [00:00<02:32,  4.44it/s]

...

100%|█████████▉| 681/682 [55:15<00:02,  2.46s/it]

Task ('drive_igor/nexus_tfrecords_raw', 'brazil', 2010, 680) finished in 54 min with state: COMPLETED


100%|██████████| 682/682 [55:15<00:00,  1.78s/it]

Task ('drive_igor/nexus_tfrecords_raw', 'brazil', 2010, 681) finished in 54 min with state: COMPLETED


100%|██████████| 682/682 [56:15<00:00,  4.95s/it]
